In [ ]:
from fastapi import FastAPI
from fastapi import Query       # used to add more details to the query parameters
from fastapi import Path        # used to add more details to the path parameters
from typing import Optional     # used to make a query parameter as optional 
from fastapi.exceptions import HTTPException
from pydantic import BaseModel  # used with the Post method 

In [ ]:
app = FastAPI()


#To run the program: uvicorn fileName.py:app --reload

###### Endpoint: An API endpoint is the point of entry in communication channel when two systems are interacting. It refers to touchpoints of the communication between an API and a server. 

### 1. GET Method: Examples    ===    ===    ===    ===    ===    ===

##### 1.1 Normal Path/Environment ---

In [ ]:
@app.get("/")
def home():
    data = {
        "message" : "Hello !!!"
    }
    return data

@app.get("/about")
def about():### 1. GET Method: Examples    ===    ===    ===    ===    ===    ===
    data = {
        "Name" : "Abhishek",
        "Organization" : "CDAC"
    }
    return data

##### 1.2 Path/Endpoint with Parameters ---

In [ ]:
car_inventory = {
    1: {
        "name" : "Nexon",
        "company" : "Tata",
        "price" : "12L"
    },
    2: {
        "name" : "Scorpio",
        "company" : "Mahindra",
        "price" : "15L"
    },
    3: {
        "name" : "Creta",
        "company" : "Hyundai",
        "price" : "14L"
    }
}


@app.get("/get-car/{car_id}")
def get_car(car_id: int):
    return car_inventory[car_id]

##### 1.3 Adding more details to path parameters eg. hints/descriptions/constraints etc. ---

In [ ]:
@app.get("/get-car/{car_id}")
def get_car(car_id: int = Path(None, description = "The ID of the Car you would like to view", gt = 0, le = 3)): # gt = greater than; lt = less than; le = less than or equals; ge = greater than or equals
    return car_inventory[car_id]

### 1.4 QUERY Parameters

comes after a question mark (?) sign in URLs usually <br>
eg: "facebook.com/home?<b>redirect</b>=/abhishek&<b>msg</b>=fail" <br>
above example has two query parameters --> redirect and msg

##### 1.4.1 Example : here [name] parameter is required*

In [ ]:
@app.get("/get-car-by-name")
def get_car_by_name(name : str):  

    for item_id in car_inventory:
        if car_inventory[item_id]["name"] == name:
            return car_inventory[item_id]
    return {"Data" : "Not Found !!"}


# access this endpoint like this: 127.0.0.1:8000/get-car-by-name?name=Creta


##### 1.4.2 Example: here [name] parameter is optional

In [ ]:
@app.get("/get-car-by-name")
def get_car_by_name(name : str = None):  #added a default value to make it optional

    for item_id in car_inventory:
        if car_inventory[item_id]["name"] == name:
            return car_inventory[item_id]
    return {"Data" : "Not Found !!"}

# access this endpoint like this: 127.0.0.1:8000/get-car-by-name

##### 1.4.3 Example: recommended way of making a parameter as Optional as per the FastAPI documentation

In [ ]:
@app.get("/get-car-by-name")
def get_car_by_name(name : Optional[str] = None):  # this is useful to give better auto-completion for the FastAPI auto documentation feature

    for item_id in car_inventory:
        if car_inventory[item_id]["name"] == name:
            return car_inventory[item_id]
    return {"Data" : "Not Found !!"}


# access this endpoint like this: 127.0.0.1:8000/get-car-by-name?name=Creta


##### 1.4.4 Example: using required* and optional* parameters together [WRONG - way]

In [ ]:
def get_car_by_name(name : Optional[str] = None, test: int):  #test is just a random parameter taken for example
    
    # above line will give an error like this : Non-default argument follows default argument 
    # because the require parameter [test] is placed after optional parameter [name]
    # for FastAPI it is fine
    # but Python generates an error like this: Non-default argument follows default argument 
    
    for item_id in car_inventory:
        if car_inventory[item_id]["name"] == name:
            return car_inventory[item_id]
    return {"Data" : "Not Found !!"}
# access this endpoint like this: 127.0.0.1:8000/get-car-by-name?name=Creta&test=1


##### 1.4.5 Example: using required* and optional* parameters together [RIGHT - way]

In [ ]:
@app.get("/get-car-by-name")
def get_car_by_name(*, name : Optional[str] = None, test: int):  #test is just a random parameter taken for example
    
    # we can add an asterisk [*] at the very beginning during function definition like above line to resolve positional parameter error
    # otherwise we have to change the positions of parameters such that
    # required parameters should appear before the optional parameters in the function definition
    # i.e like this -->  def get_car_by_name(test: int, name : Optional[str] = None):

    for item_id in car_inventory:
        if car_inventory[item_id]["name"] == name:
            return car_inventory[item_id]
    return {"Data" : "Not Found !!"}

# access this endpoint like this: 127.0.0.1:8000/get-car-by-name?name=Creta&test=1


##### 1.4.6 Example: combining PATH parameters and QUERY parameters together ---

In [ ]:
@app.get("/get-car-by-name/{car_id}")
def get_car_by_name(*, car_id: int, name : Optional[str] = None, test: int):  #test is just a random parameter taken for example

    for item_id in car_inventory:
        if car_inventory[item_id]["name"] == name:
            return car_inventory[item_id]
    return {"Data" : "Not Found !!"}


# access this endpoint like this: http://127.0.0.1:8000/get-car-by-name/1?test=1&name=Scorpio


### 2. POST Method: Examples    ===    ===    ===    ===    ===    ===

In [ ]:
# we can create a model class like this to manipulate data efficiently 
class Car (BaseModel):
    name: str
    company: str
    price: Optional[str] = None

In [ ]:
cars_data = {}   # temp dictionary to hold the data; 


#we are using this now instead of cars_inventory dict for below examples

##### 2.1 Example: adding a new item using request body of POST method

In [ ]:
@app.post("/create-item/{item_id}")
def create_item (item_id: int, item: Car):
    if item_id in cars_data:
        return {"Error" : "Item ID already exists !!"}

    cars_data[item_id] = item

    return cars_data[item_id]

###### 2.2 Example: retriving the data using GET method

In [ ]:
@app.get("/get-car-by-name")
def get_car_by_name(name : str):  

    for item_id in cars_data:
        if cars_data[item_id].name == name:
            return cars_data[item_id]
    return {"Data" : "Not Found !!"}

### 3. PUT Method: Examples    ===    ===    ===    ===    ===    ===

In [ ]:
class UpdateCar (BaseModel):
    name: Optional[str] = None
    company: Optional[str] = None
    price: Optional[str] = None
        


In [ ]:
@app.put("/update_item/{item_id}")
def update_item (item_id: int, item: UpdateCar):

    if item_id not in cars_data:
        return {"Error": "Item ID does not exists."}
    if item.name != None:
        cars_data[item_id].name = item.name 

    if item.company != None:
        cars_data[item_id].company = item.company 

    if item.price != None:
        cars_data[item_id].price = item.price 

    return cars_data[item_id]

### 4. DELETE Method: Examples    ===    ===    ===    ===    ===    ===

In [ ]:
#deleting an item by taking query parameter
@app.delete("/delete_item")
def delete_item(item_id: int = Query(..., description = "The ID of the item to delete")):
    if item_id not in cars_data:
        raise HTTPException(status_code=404, detail="Item ID does not exist.") # error codes can be supplied like this if we want 

    del cars_data[item_id]

    return {"Success" : "Item deleted !!"}

References: https://www.youtube.com/watch?v=-ykeT6kk4bk